In [1]:
## 1. Setup Google Colab

%pip install colabcode

from colabcode import ColabCode
ColabCode(port=10000)


  Using cached colabcode-0.3.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyngrok-7.3.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached nest_asyncio-1.4.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached uvicorn-0.13.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
INFO: pip is looking at multiple versions of nbclassic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of nbclassic to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ct

FileNotFoundError: [Errno 2] No such file or directory: 'wget'

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Running on CPU - this will be slow!")


CUDA available: False
Running on CPU - this will be slow!


In [8]:
# Alternative: Install in order to minimize conflicts
print("Installing dependencies in order...")

# Install core packages first
%pip install -q numpy==1.26.4
%pip install -q scikit-learn==1.6.1

# Install OpenCV packages
%pip install -q opencv-python==4.8.1.78
%pip install -q opencv-contrib-python==4.8.1.78

# Install audio processing
%pip install -q audioread==3.0.1
%pip install -q librosa==0.11.0
%pip install -q soundfile==0.13.1

# Install utilities
%pip install -q tqdm==4.67.1
%pip install -q numba==0.61.2

# Install face processing last
%pip install -q face_alignment
%pip install -q mediapipe

print("Dependencies installed!")

Installing dependencies in order...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 45.6 MB/s eta 0:00:00
Dependencies installed!


In [10]:
# Clone the official Wav2Lip repository
import os
import subprocess

if not os.path.exists('Wav2Lip'):
    subprocess.run(['git', 'clone', 'https://github.com/Rudrabha/Wav2Lip.git'])
    print("Wav2Lip repository cloned!")
else:
    print("Wav2Lip repository already exists!")

# Change to Wav2Lip directory
%cd Wav2Lip


Wav2Lip repository cloned!
/content/Wav2Lip/Wav2Lip


In [2]:
# Create checkpoints directory
os.makedirs("checkpoints", exist_ok=True)

# Download wav2lip checkpoint if not exists
checkpoint_path = "checkpoints/wav2lip.pth"
if not os.path.exists(checkpoint_path):
    print("Downloading Wav2Lip checkpoint from Hugging Face...")
    print("Source: https://huggingface.co/numz/wav2lip_studio/blob/main/Wav2lip/wav2lip.pth")

    # Use the working Hugging Face download link
    import urllib.request
    import shutil

    print("Downloading wav2lip.pth (436 MB)...")
    hf_url = "https://huggingface.co/numz/wav2lip_studio/resolve/main/Wav2lip/wav2lip.pth?download=true"

    try:
        # Download with progress indication
        with urllib.request.urlopen(hf_url) as response:
            with open(checkpoint_path, 'wb') as f:
                shutil.copyfileobj(response, f)
        print("✅ Checkpoint downloaded successfully!")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("Trying alternative method with wget...")
        try:
            subprocess.run([
                "wget", "-O", checkpoint_path,
                "https://huggingface.co/numz/wav2lip_studio/resolve/main/Wav2lip/wav2lip.pth"
            ], check=True)
            print("✅ Checkpoint downloaded with wget!")
        except Exception as e2:
            print(f"❌ Both download methods failed: {e2}")
            print("Please manually download from: https://huggingface.co/numz/wav2lip_studio/resolve/main/Wav2lip/wav2lip.pth")
else:
    print("Checkpoint already exists!")

# Verify checkpoint file
if os.path.exists(checkpoint_path):
    checkpoint_size = os.path.getsize(checkpoint_path) / (1024*1024)
    print(f"✅ Checkpoint file size: {checkpoint_size:.1f} MB")
    if checkpoint_size > 400:  # Expected size is 436 MB
        print("✅ File size looks correct!")
    else:
        print("⚠️ File size seems small, download may be incomplete")
else:
    print("❌ ERROR: Checkpoint file not found!")


Source: https://huggingface.co/numz/wav2lip_studio/blob/main/Wav2lip/wav2lip.pth
✅ Checkpoint downloaded successfully!
✅ Checkpoint file size: 415.6 MB
✅ File size looks correct!


In [6]:
from google.colab import files
import shutil

# Create directories for uploads
os.makedirs("uploads", exist_ok=True)
os.makedirs("generated/audio", exist_ok=True)

print("Upload your video and audio files:")
print("1. Face video (test_video_newscaster.mp4)")
print("2. Audio file (headline_opening_1.mp3)")

uploaded = files.upload()

# Move uploaded files to appropriate locations
for filename in uploaded.keys():
    if filename.endswith('.mp4'):
        shutil.move(filename, 'test_video_newscaster.mp4')
        print(f"Video file moved to: test_video_newscaster.mp4")
    elif filename.endswith('.mp3'):
        shutil.move(filename, 'generated/audio/headline_opening_1.mp3')
        print(f"Audio file moved to: generated/audio/headline_opening_1.mp3")
    else:
        print(f"Unknown file type: {filename}")

print("\nFile upload complete!")


Upload your video and audio files:
1. Face video (test_video_newscaster.mp4)
2. Audio file (headline_opening_1.mp3)


Saving test_video_newscaster.mp4 to test_video_newscaster (1).mp4
Saving test_audio_newscaster.mp3 to test_audio_newscaster.mp3
Video file moved to: test_video_newscaster.mp4
Audio file moved to: generated/audio/headline_opening_1.mp3

File upload complete!


In [13]:
# Check current directory and file structure
import os
print(f"Current directory: {os.getcwd()}")
print("\nFiles in current directory:")
print(os.listdir('.'))

print("\nChecking for files in different possible locations:")
possible_paths = [
    'checkpoints/wav2lip.pth',
    '../checkpoints/wav2lip.pth',
    'Wav2Lip/checkpoints/wav2lip.pth',
    'test_video_newscaster.mp4',
    '../test_video_newscaster.mp4',
    'generated/audio/headline_opening_1.mp3',
    '../generated/audio/headline_opening_1.mp3'
]

for path in possible_paths:
    exists = os.path.exists(path)
    print(f"  {path}: {'✅' if exists else '❌'}")

Current directory: /content/Wav2Lip/Wav2Lip

Files in current directory:
['hq_wav2lip_train.py', '__pycache__', 'README.md', 'preprocess.py', '.git', 'evaluation', 'hparams.py', 'temp', 'models', 'color_syncnet_train.py', 'results', 'wav2lip_train.py', 'filelists', 'face_detection', '.gitignore', 'inference.py', 'requirements.txt', 'checkpoints', 'audio.py']

Checking for files in different possible locations:
  checkpoints/wav2lip.pth: ❌
  ../checkpoints/wav2lip.pth: ✅
  Wav2Lip/checkpoints/wav2lip.pth: ❌
  test_video_newscaster.mp4: ❌
  ../test_video_newscaster.mp4: ✅
  generated/audio/headline_opening_1.mp3: ❌
  ../generated/audio/headline_opening_1.mp3: ✅


In [14]:
# Navigate to the correct directory (one level up)
import os
os.chdir('..')
print(f"Changed to directory: {os.getcwd()}")

# Verify files are now accessible
print("\nChecking files in current directory:")
files_to_check = [
    'checkpoints/wav2lip.pth',
    'test_video_newscaster.mp4',
    'generated/audio/headline_opening_1.mp3'
]

for file_path in files_to_check:
    exists = os.path.exists(file_path)
    print(f"  {file_path}: {'✅' if exists else '❌'}")

# Create results directory
os.makedirs("results", exist_ok=True)

print("\nRunning Wav2Lip inference with corrected paths...")
print("Parameters:")
print("- wav2lip_batch_size: 4")
print("- resize_factor: 2")
print("- chunk_size: 25")

# Run the inference command with correct paths
command = [
    "python", "Wav2Lip/inference.py",  # Note: inference.py is in Wav2Lip subdirectory
    "--checkpoint_path", "checkpoints/wav2lip.pth",
    "--face", "test_video_newscaster.mp4",
    "--audio", "generated/audio/headline_opening_1.mp3",
    "--outfile", "results/output_colab_test.mp4",
    "--wav2lip_batch_size", "4",
    "--resize_factor", "2"
]

print(f"Command: {' '.join(command)}")

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("✅ Inference completed successfully!")
    print("STDOUT:", result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)
except subprocess.CalledProcessError as e:
    print(f"❌ Inference failed with error: {e}")
    print("STDOUT:", e.stdout)
    print("STDERR:", e.stderr)
except Exception as e:
    print(f"❌ Unexpected error: {e}")

Changed to directory: /content/Wav2Lip

Checking files in current directory:
  checkpoints/wav2lip.pth: ✅
  test_video_newscaster.mp4: ✅
  generated/audio/headline_opening_1.mp3: ✅

Running Wav2Lip inference with corrected paths...
Parameters:
- wav2lip_batch_size: 4
- resize_factor: 2
- chunk_size: 25
Command: python Wav2Lip/inference.py --checkpoint_path checkpoints/wav2lip.pth --face test_video_newscaster.mp4 --audio generated/audio/headline_opening_1.mp3 --outfile results/output_colab_test.mp4 --wav2lip_batch_size 4 --resize_factor 2
❌ Inference failed with error: Command '['python', 'Wav2Lip/inference.py', '--checkpoint_path', 'checkpoints/wav2lip.pth', '--face', 'test_video_newscaster.mp4', '--audio', 'generated/audio/headline_opening_1.mp3', '--outfile', 'results/output_colab_test.mp4', '--wav2lip_batch_size', '4', '--resize_factor', '2']' returned non-zero exit status 1.
STDOUT: Using cpu for inference.
Reading video frames...
Number of frames available for inference: 2568
Extr

In [15]:
# Fix librosa compatibility issue
print("Fixing librosa compatibility issue...")

# Uninstall current librosa
%pip uninstall -y librosa

# Install the specific version that works with Wav2Lip
%pip install -q librosa==0.8.1

print("librosa version fixed!")

# Now try the inference again
print("\nRunning Wav2Lip inference with fixed librosa...")

# Run the inference command with correct paths
command = [
    "python", "Wav2Lip/inference.py",
    "--checkpoint_path", "checkpoints/wav2lip.pth",
    "--face", "test_video_newscaster.mp4",
    "--audio", "generated/audio/headline_opening_1.mp3",
    "--outfile", "results/output_colab_test.mp4",
    "--wav2lip_batch_size", "4",
    "--resize_factor", "2"
]

print(f"Command: {' '.join(command)}")

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("✅ Inference completed successfully!")
    print("STDOUT:", result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)
except subprocess.CalledProcessError as e:
    print(f"❌ Inference failed with error: {e}")
    print("STDOUT:", e.stdout)
    print("STDERR:", e.stderr)
except Exception as e:
    print(f"❌ Unexpected error: {e}")

Fixing librosa compatibility issue...
Found existing installation: librosa 0.11.0
Uninstalling librosa-0.11.0:
  Successfully uninstalled librosa-0.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.8 MB/s eta 0:00:00
librosa version fixed!

Running Wav2Lip inference with fixed librosa...
Command: python Wav2Lip/inference.py --checkpoint_path checkpoints/wav2lip.pth --face test_video_newscaster.mp4 --audio generated/audio/headline_opening_1.mp3 --outfile results/output_colab_test.mp4 --wav2lip_batch_size 4 --resize_factor 2
❌ Inference failed with error: Command '['python', 'Wav2Lip/inference.py', '--checkpoint_path', 'checkpoints/wav2lip.pth', '--face', 'test_video_newscaster.mp4', '--audio', 'generated/audio/headline_opening_1.mp3', '--outfile', 'results/output_colab_test.mp4', '--wav2lip_batch_size', '4', '--resize_factor', '2']' returned non-zero exit status 1.
STDOUT: 
STDERR: Traceback (mo

In [ ]:
# List output files
print("Output files:")
if os.path.exists("results"):
    for file in os.listdir("results"):
        print(f"  {file}")

# Check if output files exist and get their info
output_files = [
    "results/output_colab_test.mp4",
    "result_voice.mp4"  # Default output name
]

import cv2

for output_file in output_files:
    if os.path.exists(output_file):
        file_size = os.path.getsize(output_file) / (1024*1024)
        print(f"\n✅ Output found: {output_file}")
        print(f"   Size: {file_size:.1f} MB")

        # Get video info
        try:
            cap = cv2.VideoCapture(output_file)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            duration = frame_count / fps if fps > 0 else 0
            cap.release()

            print(f"   Resolution: {width}x{height}")
            print(f"   FPS: {fps}")
            print(f"   Duration: {duration:.2f} seconds")
            print(f"   Frame count: {frame_count}")
        except Exception as e:
            print(f"   Error reading video info: {e}")
    else:
        print(f"❌ Output not found: {output_file}")

# Download the output file if it exists
download_files = [f for f in output_files if os.path.exists(f)]

if download_files:
    print(f"\nDownloading {len(download_files)} output file(s)...")
    for file_path in download_files:
        print(f"Downloading: {file_path}")
        files.download(file_path)
        print(f"✅ Downloaded: {file_path}")

    print("\n🎉 Wav2Lip inference test complete!")
    print("✨ Check the downloaded videos for results.")
else:
    print("\n❌ No output files found to download!")
    print("Please check the inference output above for errors.")


In [ ]:
# Alternative: Test with enhanced chunked inference (if available)
print("Testing with chunked parameters...")
print("This requires a custom inference script with chunking support.")

# Check if chunked inference script exists
chunked_scripts = [
    "inference_chunked.py",
    "../wav2lip_test/Wav2Lip/inference_chunked.py"
]

chunked_script = None
for script in chunked_scripts:
    if os.path.exists(script):
        chunked_script = script
        break

if chunked_script:
    print(f"Found chunked script: {chunked_script}")

    # Run with chunked parameters
    chunked_command = [
        "python", chunked_script,
        "--checkpoint_path", "checkpoints/wav2lip.pth",
        "--face", "test_video_newscaster.mp4",
        "--audio", "generated/audio/headline_opening_1.mp3",
        "--outfile", "results/output_chunked_test.mp4",
        "--wav2lip_batch_size", "4",
        "--resize_factor", "2",
        "--chunk_size", "25"
    ]

    print(f"Chunked command: {' '.join(chunked_command)}")

    try:
        result = subprocess.run(chunked_command, capture_output=True, text=True, check=True)
        print("✅ Chunked inference completed successfully!")
        print("STDOUT:", result.stdout)
        if result.stderr:
            print("STDERR:", result.stderr)

        # Download chunked result if it exists
        if os.path.exists("results/output_chunked_test.mp4"):
            files.download("results/output_chunked_test.mp4")
            print("✅ Downloaded chunked result!")

    except subprocess.CalledProcessError as e:
        print(f"❌ Chunked inference failed: {e}")
        print("STDOUT:", e.stdout)
        print("STDERR:", e.stderr)
else:
    print("❌ No chunked inference script found.")
    print("Using standard inference.py instead.")
    print("For chunked processing, you would need to:")
    print("1. Upload your custom inference_chunked.py script")
    print("2. Or modify the existing inference.py to support chunking")
